In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,),(0.3081,))
                          ])
                         ),
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,),(0.3081,))
                          ])
                         ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolution
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        
        # Dropout
        self.drop = nn.Dropout2d()
        
        # Fully Connected layer
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = x.view(-1, 320) # 2차원의 feature map을 1차원으로 펴줌
        
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [10]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [12]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset), 100.*batch_idx/len(train_loader), loss.item()))

In [13]:
def evaluate(model, test_loader):
    model.eval()
    
    test_loss=0
    correct=0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [14]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:2.304023
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.223998
Train Epoch: 1 [25600/60000 (43%)]	Loss:0.930845
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.804982
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.900487
[1] Test Loss: 0.6389, Accuracy: 75.83%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.929037
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.777151
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.719654
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.646240
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.791094
[2] Test Loss: 0.5438, Accuracy: 78.85%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.674471
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.738865
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.566235
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.647510
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.579845
[3] Test Loss: 0.4922, Accuracy: 81.65%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.587617
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.577441
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.615582
Train Epoch: 4 [38

Train Epoch: 29 [51200/60000 (85%)]	Loss:0.340168
[29] Test Loss: 0.2921, Accuracy: 89.33%
Train Epoch: 30 [0/60000 (0%)]	Loss:0.292029
Train Epoch: 30 [12800/60000 (21%)]	Loss:0.319178
Train Epoch: 30 [25600/60000 (43%)]	Loss:0.334005
Train Epoch: 30 [38400/60000 (64%)]	Loss:0.279883
Train Epoch: 30 [51200/60000 (85%)]	Loss:0.322826
[30] Test Loss: 0.2974, Accuracy: 89.46%
Train Epoch: 31 [0/60000 (0%)]	Loss:0.366283
Train Epoch: 31 [12800/60000 (21%)]	Loss:0.175913
Train Epoch: 31 [25600/60000 (43%)]	Loss:0.233806
Train Epoch: 31 [38400/60000 (64%)]	Loss:0.381304
Train Epoch: 31 [51200/60000 (85%)]	Loss:0.214551
[31] Test Loss: 0.2920, Accuracy: 89.20%
Train Epoch: 32 [0/60000 (0%)]	Loss:0.167865
Train Epoch: 32 [12800/60000 (21%)]	Loss:0.285503
Train Epoch: 32 [25600/60000 (43%)]	Loss:0.270993
Train Epoch: 32 [38400/60000 (64%)]	Loss:0.190672
Train Epoch: 32 [51200/60000 (85%)]	Loss:0.349509
[32] Test Loss: 0.2883, Accuracy: 89.42%
Train Epoch: 33 [0/60000 (0%)]	Loss:0.254939
Train 